In [72]:
!pip install -q langchain_groq langchain-core requests dotenv

In [73]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_token = os.getenv("test_groq")
llm = ChatGroq(
    groq_api_key=groq_token,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)

# Use llm directly - no need for ChatHuggingFace wrapper
response = llm.invoke("hi")
print(response)

content="Hello! How can I assist you today? Do you have a specific question, topic you'd like to discuss, or perhaps need help with something? I'm here to help!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 11, 'total_tokens': 46, 'completion_time': 0.091561476, 'prompt_time': 0.002881411, 'queue_time': 0.28216281, 'total_time': 0.094442887}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--01b3e377-6baf-4dea-9e04-27e5faa5943a-0' usage_metadata={'input_tokens': 11, 'output_tokens': 35, 'total_tokens': 46}


In [74]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [75]:
from langchain_core.tools import tool, InjectedToolArg
from pydantic import BaseModel
from typing import Annotated
#tool create

@tool
def get_conversion_factor(base_currency:str,target_currency:str)->float:
  """
  this functiom fetches the currency conversion factor between given base currency
  and a target currency
  """
  api=os.getenv("exchange_factor")
  url = f'https://v6.exchangerate-api.com/v6/{api}/pair/{base_currency}/{target_currency}'

  response=requests.get(url)
  return response.json()

@tool
def convert(base_currency_value:int,conversion_rate:Annotated[float,InjectedToolArg])->float:

  """
  given a currency conversion rate this function calculates the target currency
  value
  """

  return base_currency_value*conversion_rate


In [76]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754179202,
 'time_last_update_utc': 'Sun, 03 Aug 2025 00:00:02 +0000',
 'time_next_update_unix': 1754265602,
 'time_next_update_utc': 'Mon, 04 Aug 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.4485}

In [77]:
convert.invoke({'base_currency_value':10,'conversion_rate':87.4485})

874.4849999999999

In [78]:
#tool binding

llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [79]:
#Tool calling

query=HumanMessage("""What is the conversion rate between USD and INR ,
                      and based on that can you convert 10 USD to INR

                      "Convert 10 USD to INR. Ensure `base_currency_value` is an integer, not a string.""")
messages=[query]

In [80]:
messages

[HumanMessage(content='What is the conversion rate between USD and INR ,\n                      and based on that can you convert 10 USD to INR\n                      \n                      "Convert 10 USD to INR. Ensure `base_currency_value` is an integer, not a string.', additional_kwargs={}, response_metadata={})]

In [81]:
ai_message=llm_with_tools.invoke(messages)
ai_message
messages.append(ai_message)

In [82]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'hnf4rek49',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'srjamkhrp',
  'type': 'tool_call'}]

In [83]:
import json

In [84]:
for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [85]:
messages

[HumanMessage(content='What is the conversion rate between USD and INR ,\n                      and based on that can you convert 10 USD to INR\n                      \n                      "Convert 10 USD to INR. Ensure `base_currency_value` is an integer, not a string.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hnf4rek49', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'srjamkhrp', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 781, 'total_tokens': 834, 'completion_time': 0.126694353, 'prompt_time': 0.059200639, 'queue_time': 0.19468332300000002, 'total_time': 0.185894992}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand',

In [86]:
llm_with_tools.invoke(messages)

AIMessage(content='The conversion rate between USD and INR is 87.4485. \n10 USD is equal to 874.485 INR.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 1000, 'total_tokens': 1027, 'completion_time': 0.066148129, 'prompt_time': 0.05139063, 'queue_time': 0.201092488, 'total_time': 0.117538759}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d43f9a94-7279-47ba-9189-e1d253b7507b-0', usage_metadata={'input_tokens': 1000, 'output_tokens': 27, 'total_tokens': 1027})

In [87]:
llm_with_tools.invoke(messages).content

'The conversion rate between USD and INR is 87.4485. \n\n10 USD is equal to 874.485 INR.'

In [88]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [89]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: I'm just a language model, I don't have feelings like humans do, but I'm functioning properly and ready to help with any questions or tasks you have. 

Action:
```
{
 "action": "Final Answer",
 "action_input": "I'm doing well, thanks for asking! How can I assist you today?"
}
```

> Finished chain.
